In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("influencers.csv")
data.head()

,id,followerCount,engagementRate
0,1,6207,1.0
1,2,38237,5.8
2,3,14840,0.6
3,4,43851,1.4
4,5,15530,3.5


In [3]:
class Datafuzzy():
    def __init__(self, score, decission):
        self.score = float(score)
        self.decission = decission

In [4]:
# FOLLOWER
# membership function
def fuzzyFollower(countFol):

    follower = []
    # STABLE GRAPH
    if (0 <= countFol and countFol <=20000):
        scoreFuzzy = 1
        follower.append(Datafuzzy(scoreFuzzy, "NANO"))
      
    # GRAPH DOWN
    elif (20000 < countFol and countFol <= 30000):
        scoreFuzzy = np.absolute((countFol - 20000) / (30000 - 20000))
        follower.append(Datafuzzy("%.2f" % scoreFuzzy, "NANO"))
      
    # MICRO
    # THE GRAPH GOES UP
    if (20000 <= countFol and countFol <= 30000):
        scoreFuzzy = 1 - scoreFuzzy #np.absolute((countFol - 30000) / (30000 - 20000) )
        follower.append(Datafuzzy("%.2f" % scoreFuzzy, "MICRO"))
      
    # STABLE GRAPH
    elif (30000 < countFol and countFol < 55000):
        scoreFuzzy = 1
        follower.append(Datafuzzy(scoreFuzzy, "MICRO"))
      
    # GRAPH DOWN
    elif (55000 <= countFol and countFol <= 75000):
        scoreFuzzy = np.absolute((countFol - 55000) / (75000 - 55000))
        follower.append(Datafuzzy("%.2f" % scoreFuzzy, "MICRO"))

    # MEDIUM
    # THE GRAPH GOES UP
    if (55000 <= countFol and countFol <= 75000):
        scoreFuzzy = 1 - scoreFuzzy
        follower.append(Datafuzzy("%.2f" % scoreFuzzy, "MEDIUM"))
      
  # STABLE GRAPH
    elif (75000 < countFol and countFol <= 80000):
        scoreFuzzy = 1
        follower.append(Datafuzzy(scoreFuzzy, "MEDIUM"))
        
    elif (countFol > 80000):
        scoreFuzzy = np.absolute((countFol - 80000) / (100000 - 80000))
        follower.append(Datafuzzy(scoreFuzzy, "MEDIUM"))
    
    return follower

In [5]:
# ENGAGEMENT RATE 
# membership function
def fuzzyEngagement(countEng):

    engagement = []
    # STABLE GRAPH
    if (0 <= countEng and countEng <= 0.3):
        scoreFuzzy = 1
        engagement.append(Datafuzzy(scoreFuzzy, "NANO"))
      
    # GRAPH DOWN
    elif (0.3 < countEng and countEng <= 1.7):
        scoreFuzzy = np.absolute((countEng - 0.31) / (1.7 - 0.31))
        engagement.append(Datafuzzy("%.2f" % scoreFuzzy, "NANO"))
      
    # MICRO
    # THE GRAPH GOES UP
    if (0.3 < countEng and countEng <= 1.7):
        scoreFuzzy =  1 - scoreFuzzy # np.absolute((countEng - 0.3) / (1.7 - 0.3))
        engagement.append(Datafuzzy("%.2f" % scoreFuzzy, "MICRO"))

    # GRAPH DOWN
    elif (1.7 < countEng and countEng <= 5.7):
        scoreFuzzy = np.absolute((countEng - 1.71) / (5.7 - 1.71))
        engagement.append(Datafuzzy("%.2f" % scoreFuzzy, "MICRO"))

    # MEDIUM
    # THE GRAPH GOES UP
    if (1.7 < countEng and countEng <= 5.7):
        scoreFuzzy = 1 - np.absolute((countEng - 1.71) / (5.7 - 1.71))
        engagement.append(Datafuzzy("%.2f" % scoreFuzzy, "MEDIUM"))
      
    # STABLE GRAPH
    elif (5.71 < countEng and countEng <= 10.0):
        scoreFuzzy = 1
        engagement.append(Datafuzzy(scoreFuzzy, "MEDIUM"))
        
    return engagement


In [6]:
# Fuzzy Rules
def fuzzyRules(follower, engagement):
    temp_yes = []
    temp_no = []
    
    # First decision Follower test is Nano
    if (follower[0].decission == "NANO"):
        # Get minimal score fuzzy every decision NO or YES
        temp_yes.append(min(follower[0].score,engagement[0].score))
        
        # if get 2 data fuzzy Engagement
        if (len(engagement) > 1):
            temp_yes.append(min(follower[0].score,engagement[1].score))
            
    # First decision of Follower is Micro or Medium                     
    else:
        if (engagement[0].decission == "NANO"):
            temp_no.append(min(follower[0].score, engagement[0].score))
            
        elif (engagement[0].decission == "MICRO"):
            if (follower[0].decission == "MICRO"):
                temp_yes.append(min(follower[0].score, engagement[0].score))
            else:
                temp_no.append(min(follower[0].score,engagement[0].score))
                
        else:
            temp_yes.append(min(follower[0].score, engagement[0].score))
            
        # if get 2 data fuzzy engagement 
        if (len(engagement) > 1):
            if (engagement[1].decission == "NANO"):
                temp_no.append(min(follower[0].score, engagement[1].score))
                
            elif (engagement[0].decission == "MICRO"):
                if (follower[0].decission == "MICRO"):
                    temp_yes.append(min(follower[0].score, engagement[1].score))
                else:
                    temp_no.append(min(follower[0].score,engagement[1].score))
                    
            else:
                temp_yes.append(min(follower[0].score, engagement[1].score))
                
    # if get 2 data fuzzy Follower                    
    if (len(follower) > 1):
        # Second decision Follower is Nano
        if (follower[1].decission == "NANO"):
            temp_yes.append(min(engagement[0].score,follower[1].score))  
            
            if (len(engagement) > 1):
                temp_yes.append(min(engagement[1].score,follower[1].score))
                
        # Second decision follower is Micro or Medium
        else:
            if (engagement[0].decission == "NANO"):
                temp_no.append(min(follower[1].score, engagement[0].score))
                
            elif (engagement[0].decission == "MICRO"):
                if (follower[1].decission == "MICRO"):
                    temp_yes.append(min(follower[1].score, engagement[0].score))
                else:
                    temp_no.append(min(follower[1].score,engagement[0].score))
                    
            #if get 2 data fuzzy Engagement 
            if (len(engagement) > 1):
                if (engagement[1].decission == "NANO"):
                    temp_no.append(min(follower[1].score, engagement[1].score))
                    
                elif (engagement[1].decission == "MICRO"):
                    if (follower[1].decission == "MICRO"):
                        temp_yes.append(min(follower[1].score, engagement[1].score))
                    else:
                        temp_no.append(min(follower[1].score,engagement[1].score))
    
    return temp_yes, temp_no

In [7]:
# Result
def getResult(resultYes, resultNo):
    yes = 0
    no = 0
    if(not resultYes and not resultNo):
        print("NO and YES is Error")
    else:
        if(resultNo):
            if (len(resultNo) > 1):
                no = max(resultNo)
            else:
                no = float(resultNo[0])

        if(resultYes):
            if(len(resultYes) > 1):
                yes = max(resultYes)
            else:
                yes = float(resultYes[0])
    return yes, no


In [8]:
def finalDecission(yes, no):
    sugeno = ((no * 60) + (yes * 80)) / (no + yes)
    return sugeno

In [9]:
def mainFunction(followerCount, engagementRate):
    follower = fuzzyFollower(followerCount)
    engagement = fuzzyEngagement(engagementRate)
    
    resultYes, resultNo = fuzzyRules(follower, engagement)
    
    yes, no = getResult(resultYes, resultNo)
    
    return finalDecission(yes, no)



In [14]:
 # MAIN
hasil = []
for i in range (len(data)):
    hasil.append([data.loc[i, 'id'], mainFunction(data.loc[i, 'followerCount'], data.loc[i, 'engagementRate'])])
hasil.sort(key=lambda x:x[1], reverse=True)
print("HASIL")
print(*hasil, sep='\n')

chosen = pd.DataFrame(hasil[:20], columns=['ID', 'Score'])
chosen.to_csv('choosen.csv')

HASIL
[14, 80.00000000000001]
[1, 80.0]
[2, 80.0]
[3, 80.0]
[5, 80.0]
[6, 80.0]
[8, 80.0]
[9, 80.0]
[10, 80.0]
[11, 80.0]
[12, 80.0]
[13, 80.0]
[15, 80.0]
[17, 80.0]
[20, 80.0]
[22, 80.0]
[23, 80.0]
[24, 80.0]
[27, 80.0]
[28, 80.0]
[29, 80.0]
[30, 80.0]
[31, 80.0]
[32, 80.0]
[34, 80.0]
[35, 80.0]
[36, 80.0]
[39, 80.0]
[40, 80.0]
[44, 80.0]
[45, 80.0]
[48, 80.0]
[50, 80.0]
[51, 80.0]
[54, 80.0]
[55, 80.0]
[57, 80.0]
[58, 80.0]
[59, 80.0]
[61, 80.0]
[64, 80.0]
[67, 80.0]
[70, 80.0]
[71, 80.0]
[73, 80.0]
[74, 80.0]
[76, 80.0]
[77, 80.0]
[78, 80.0]
[80, 80.0]
[82, 80.0]
[83, 80.0]
[84, 80.0]
[85, 80.0]
[86, 80.0]
[87, 80.0]
[88, 80.0]
[89, 80.0]
[90, 80.0]
[91, 80.0]
[92, 80.0]
[93, 80.0]
[95, 80.0]
[96, 80.0]
[97, 80.0]
[100, 80.0]
[43, 79.45945945945945]
[42, 79.39999999999999]
[38, 78.79999999999998]
[62, 78.79999999999998]
[65, 78.79999999999998]
[79, 78.79999999999998]
[98, 78.5]
[94, 77.2]
[68, 76.6]
[33, 75.8]
[21, 72.92929292929293]
[16, 71.6]
[47, 71.6]
[66, 71.6]
[18, 70.0]
[52, 